## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
from keras import backend as K
from keras.datasets import cifar10
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import tensorflow as tf
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [3]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
batch_size = 64 # batch 的大小
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量

In [5]:
model = ResNet50(weights='imagenet', include_top=False,input_shape=(32,32,3))
x = model.output
x = Flatten()(x)
x = Dropout(0.2)(x)
output = Dense(num_classes, activation='softmax', name='softmax')(x)
net_final = Model(inputs=model.input, outputs=output)
net_final.summary()

onv[0][0]        
__________________________________________________________________________________________________
conv4_block1_1_relu (Activation (None, 2, 2, 256)    0           conv4_block1_1_bn[0][0]          
__________________________________________________________________________________________________
conv4_block1_2_conv (Conv2D)    (None, 2, 2, 256)    590080      conv4_block1_1_relu[0][0]        
__________________________________________________________________________________________________
conv4_block1_2_bn (BatchNormali (None, 2, 2, 256)    1024        conv4_block1_2_conv[0][0]        
__________________________________________________________________________________________________
conv4_block1_2_relu (Activation (None, 2, 2, 256)    0           conv4_block1_2_bn[0][0]          
__________________________________________________________________________________________________
conv4_block1_0_conv (Conv2D)    (None, 2, 2, 1024)   525312      conv3_block4_out[0][0]    

In [6]:
net_final.compile(optimizer=Adam(),loss='categorical_crossentropy', metrics=['accuracy'])
#ImageDataGenerator
augment_generator = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

history = net_final.fit_generator(
                    augment_generator.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=int(len(x_train)/batch_size),
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
781/781 [==============================] - 2282s 3s/step - loss: 2.5791 - accuracy: 0.1205 - val_loss: 2.3911 - val_accuracy: 0.0948
Epoch 2/10
781/781 [==============================] - 2288s 3s/step - loss: 2.6685 - accuracy: 0.1110 - val_loss: 2.3029 - val_accuracy: 0.0949
Epoch 3/10
781/781 [==============================] - 2290s 3s/step - loss: 2.5312 - accuracy: 0.1155 - val_loss: 2.3164 - val_accuracy: 0.1014
Epoch 4/10
781/781 [==============================] - 2289s 3s/step - loss: 2.3705 - accuracy: 0.1223 - val_loss: 2.3727 - val_accuracy: 0.0993
Epoch 5/10
781/781 [==============================] - 2275s 3s/step - loss: 2.2749 - accuracy: 0.1326 - val_loss: 2.3275 - val_accuracy: 0.0798
Epoch 6/10
781/781 [==============================] - 2276s 3s/step - loss: 2.2700 - accuracy: 0.1359 - val_loss: 2.3354 - val_accuracy: 0.1329
Epoch 7/10
781/781 [==============================] - 2280s 

In [7]:
score = net_final.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 17s 55ms/step - loss: 2.4289 - accuracy: 0.1459
Test loss: 2.4288926124572754
Test accuracy: 0.14589999616146088
